In [1]:
# -------- Code Outline -------- #
# Code calculates the evaluations and the predictions from
# The saved RNN model. (preferably do this section on a high powered
# Computer as it takes a while)

In [2]:
# Used to open pickle files and plot figures
import pickle
import matplotlib.pyplot as plt
# These all used to create neural network
import keras
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, GlobalMaxPool1D, Dense, Dropout
import tensorflow as tf
# These used to open all the files at once into one large list of arrays
import os
import numpy as np

Using TensorFlow backend.


In [3]:
# -------- Loader -------- #
# First we need to input the model and weights and re-create it
# As well as loading in the test data and the training data

# Load in training data
with open('data sets/Window_80_Training_Set_x.pkl', 'rb') as f:
    train_x = pickle.load(f)
with open('data sets/Window_80_Training_Set_y.pkl', 'rb') as f:
    train_y = pickle.load(f)
    
# Load in test data
with open('data sets/Window_80_Test_Set_x.pkl', 'rb') as f:
    test_x = pickle.load(f)
with open('data sets/Window_80_Test_Set_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

In [4]:
train_x.shape
train_y.shape

(774144,)

In [5]:
# -------- Model -------- #
# We copy over the model function from the RNN code so that we can replicate
# A prediction model version

def createModel(batch_size, time_steps, features):
    initializer = "glorot_uniform" # xavier normal
    model = Sequential();
    # output shape = (batch_size, timesteps, units)
    # batch size specifies the number of SUBSEQUENT windows passed per training step
    model.add(Bidirectional(LSTM(units=200,
                                 return_sequences=True, # Many to many layer output
                                 stateful=False, # Subsequent batches are not independant
                                 # Reinitalize subsequent training steps with previous information
                                 # Overcomes memory limits in feeding in data
                                 recurrent_dropout=0.1,
                                 activation='sigmoid',
                                 bias_initializer=initializer,
                                 kernel_initializer=initializer),
                                 merge_mode="concat", batch_input_shape = (batch_size, time_steps, features))) # LSTM Dropout
    model.add(GlobalMaxPool1D())#input_shape=(400, 100)))
    model.add(Dense(units=50, bias_initializer=initializer, kernel_initializer=initializer, activation="relu", input_shape=(100, 400)))
    model.add(Dropout(rate=0.1))
    model.add(Dense(units=1, activation="sigmoid", bias_initializer=initializer, kernel_initializer=initializer))
    return model

In [6]:
# -------- Create Model -------- #
# This model is identical to the RNN model however we now have the batch
# Size set to 1 for prediction and evaluation

train_model = createModel(1, 80, 1)
print(train_model.summary())
# Use Adam optimizer as per paper
opt = keras.optimizers.Adam(lr=0.001)  #, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0, amsgrad=False)
# Binary_crossentropy as per paper - because binary classification
train_model.compile(optimizer=opt, loss = 'binary_crossentropy', metrics=["binary_accuracy", "mae"])#metrics = ['mse', 'mae', 'accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (1, 80, 400)              323200    
_________________________________________________________________
global_max_pooling1d_1 (Glob (1, 400)                  0         
_________________________________________________________________
dense_1 (Dense)              (1, 50)                   20050     
_________________________________________________________________
dropout_1 (Dropout)          (1, 50)                   0         
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    51        
Total params: 343,301
Trainable params: 343,301
Non-trainable params: 0
_________________________________________________________________
None


Instructio

In [7]:
# Now load in the model weights after creating an identical model

train_model.load_weights("saved_model/model_weight-80.h5")

In [8]:
# -------- Predictions -------- #
# We need the prediction array so that we can calculate sensitivies and specificities

# Now we have the model we can evaluate on test and training set

# First predict on test set
# predictions = train_model.predict(test_x, batch_size = 1, verbose = True)

In [11]:
# -------- Output the predictions -------- #

# filename = 'Window_80_predictions'
# This line uses pickle to save the window arrays as a .pkl file

# with open('{}.pkl'.format(filename), 'wb') as f:
#      pickle.dump(predictions, f)

In [15]:
# -------- Evaluator -------- #
# Evaluate on both the test and the training set to find out the
# Overall loss and metrics for the model

# Now evaluate on test set
train_model.evaluate(test_x, test_y, batch_size = 1, verbose = True)

258401/258401 [==============================] - 6838s 26ms/step


[0.0031559535639030375, 0.9988970630918611, 0.0018881784765436493]

In [16]:
# Evaluate on the training set (final check)
train_model.evaluate(train_x, train_y, batch_size = 1, verbose = True)

774144/774144 [==============================] - 20167s 26ms/step


[0.002377908284366665, 0.9992171999007936, 0.0015763516335619654]